In [18]:
import joblib
import pywt
import cv2
import numpy as np
import json

In [3]:
def crop_image(path):
    result = []
    
    image = cv2.imread(path)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    face_cascade = cv2.CascadeClassifier("./opencv/haarcascades/haarcascade_frontalface_default.xml")
    eye_cascade = cv2.CascadeClassifier("./opencv/haarcascades/haarcascade_eye.xml")
    
    faces = face_cascade.detectMultiScale(image_gray, 1.3, 5)
    
    for (x, y, w, h) in  faces:
        face_region = image[y : y+h, x : x+w]
        face_region_gray = image_gray[y : y+h, x : x+w]
        
        eyes = eye_cascade.detectMultiScale(face_region_gray, 1.3, 5)
        
        if len(eyes) >= 2 :
            result.append(face_region)

    return result

In [9]:
def transform(img, mode='haar', level=1):
    imArray = img
    imArray = cv2.cvtColor(imArray,cv2.COLOR_RGB2GRAY)
    imArray =  np.float32(imArray)   
    imArray /= 255;
    coeffs=pywt.wavedec2(imArray, mode, level=level)
    coeffs_H=list(coeffs)  
    coeffs_H[0] *= 0;  
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H = np.uint8(imArray_H)
    return imArray_H

In [33]:
RESIZE_SIZE = 48

def classifier(image_path, model, classes):
    faces = crop_image(image_path)
    
    print(f"Faces Detected: {len(faces)}")
    count = 1
    
    for face in faces:
        scaled_image = cv2.resize(face, (RESIZE_SIZE, RESIZE_SIZE))
        wavelet_image = transform(face, 'db1', 5)
        scaled_wavelet_image = cv2.resize(wavelet_image, (RESIZE_SIZE, RESIZE_SIZE))
        combined_image = np.vstack((
                scaled_image.reshape(RESIZE_SIZE * RESIZE_SIZE * 3, 1), 
                scaled_wavelet_image.reshape(RESIZE_SIZE*RESIZE_SIZE, 1)
        ))
        
        X = [combined_image]
        X = np.array(X).reshape(len(X), X[0].shape[0]).astype(float)
        
        prediction = model.predict(X)[0]
        
        print(f"Face {count}: {classes[prediction]}")
        
        count += 1

In [22]:
model = None
name_label_dict = None

with open("model.pkl", "rb") as f:
    model = joblib.load(f)
    
with open("classes.json", "r") as f:
    name_label_dict = json.load(f)
    name_label_dict = { v : k for k, v in name_label_dict.items() }

In [23]:
name_label_dict

{0: 'Bhuvneshwar Kumar',
 1: 'Dinesh Karthik',
 2: 'Hardik Pandya',
 3: 'K L Rahul',
 4: 'Kedar Jadhav',
 5: 'M S Dhoni',
 6: 'Rohit Sharma',
 7: 'Shikhar Dhawan',
 8: 'Virat Kohli',
 9: 'Yuzvendra Chahal'}

In [36]:
classifier("./test_images/6.jpeg", model, name_label_dict)

Faces Detected: 1
Face 1: M S Dhoni
